## Plotting Vessel Trail with Signals API

This notebook will mainly cover how to extract signals data from Signals API and how to visualize them in a map using plotly library.

The example used in this notebook is LNG Exports out of Russia Arctic LNG 2 Plant terminal.

### Import Library

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import vortexasdk as v
import requests
import os
import plotly.express as px

c:\Users\OuiWeinJien\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
lng = [p.id for p in v.Products().search('LNG').to_list() if p.name=='LNG']

russia_arctic=[p.id for p in v.Geographies().search('Arctic LNG 2 Plant').to_list()]

2024-09-17 15:13:57,286 vortexasdk.client — WARNING — You are using vortexasdk version 0.73.0, however version 0.73.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


### Extract Cargo Movements data

In [3]:
cm_df = v.CargoMovements().search(
    filter_activity='loading_end',
    filter_products=lng,
    filter_origins=russia_arctic,
    filter_time_min=datetime(2024, 1, 1),
    filter_time_max=datetime.today()
).to_df(columns = 'all')

In [4]:
cm_df.head(5)

,cargo_movement_id,quantity,status,vessels.0.dwt,vessels.0.fixture_id,vessels.0.flag.0.flag,vessels.0.flag.0.flag_country,vessels.0.flag.0.tag,vessels.0.mmsi,vessels.0.start_timestamp,...,events.cargo_waypoint_event.8.start_timestamp,events.cargo_waypoint_event.8.probability,events.cargo_waypoint_event.8.vessel_id,events.cargo_waypoint_event.8.location.waypoint.probability,events.cargo_waypoint_event.8.location.waypoint.id,events.cargo_waypoint_event.8.location.waypoint.label,events.cargo_waypoint_event.8.location.waypoint.source,events.cargo_waypoint_event.8.location.waypoint.layer,parent_ids.0.splinter_timestamp,parent_ids.0.id
0,a9486b756c4994b3fccbe9b66bec144ea9486b756c4994...,433936,transiting_state,77410,,PW,479de61e68a1f232b16498c726ae6afb6de766da18472e...,vessel_flag_tag,511101456,2024-09-04 00:00:00+00:00,...,NaT,,,,,,,,NaT,
1,ad225abb9cf80f7a32e4eca229e1ca59ad225abb9cf80f...,833796,transiting_state,79866,,PW,479de61e68a1f232b16498c726ae6afb6de766da18472e...,vessel_flag_tag,511101428,2024-08-07 00:00:00+00:00,...,NaT,,,,,,,,NaT,
2,c38b5ca7177a343a613fb60de51a1179c38b5ca7177a34...,433936,transiting_state,77410,,PW,479de61e68a1f232b16498c726ae6afb6de766da18472e...,vessel_flag_tag,511101456,2024-08-23 08:00:00+00:00,...,NaT,,,,,,,,NaT,
3,c920f7d026d99a8424b9346e59531810c920f7d026d99a...,190805,transiting_state,77712,,PW,479de61e68a1f232b16498c726ae6afb6de766da18472e...,vessel_flag_tag,511101396,2024-08-01 00:00:00+00:00,...,NaT,,,,,,,,NaT,
4,fb8b2b93e566c43dfed56a5410bbc8e7fb8b2b93e566c4...,676890,transiting_state,77712,,PW,479de61e68a1f232b16498c726ae6afb6de766da18472e...,vessel_flag_tag,511101396,2024-08-01 00:00:00+00:00,...,2024-09-16 09:49:32+00:00,1.0,3bc8490a0c382976f8746101508e709cb3f69d2a0efc5b...,1.0,703a4af533b42e86a0473f6b464ad2da1bb04cf899a167...,Cape Finisterre,model,waypoint,2024-09-11 19:06:06+00:00,c920f7d026d99a8424b9346e59531810c920f7d026d99a...


In [93]:
cm_df[['vessels.0.name','vessels.0.imo','vessels.0.id','vessels.1.name','vessels.1.imo','vessels.1.start_timestamp']]

,vessels.0.name,vessels.0.imo,vessels.0.id,vessels.1.name,vessels.1.imo,vessels.1.start_timestamp
0,EVEREST ENERGY,9243148,fe16241b4f45168faa94269633b52a4336b0c9d6e4c76b...,,,NaT
1,ASYA ENERGY,9216298,79138d1707099e3ed42779a5249bdf12343e0acdc53d85...,,,NaT
2,EVEREST ENERGY,9243148,fe16241b4f45168faa94269633b52a4336b0c9d6e4c76b...,,,NaT
3,PIONEER,9256602,72674aadb46c52864d2fa32ef49361f6873fd11c69574c...,,,NaT
4,PIONEER,9256602,72674aadb46c52864d2fa32ef49361f6873fd11c69574c...,NEW ENERGY,9324277.0,2024-08-22 00:00:00+00:00


In [5]:
vessels_dict = {'fe16241b4f45168f': 'EVEREST ENERGY',
 '79138d1707099e3e': 'ASYA ENERGY',
 '72674aadb46c5286': 'PIONEER',
 '3bc8490a0c382976': 'NEW ENERGY'}

In [8]:
all_list = ['9243148', '9216298', '9243148', '9256602', '9256602', '9324277']

api_key = os.environ['VORTEXA_API_KEY']

start_timestamp = pd.to_datetime(min(min(cm_df['vessels.0.start_timestamp'].values),min(cm_df['vessels.1.start_timestamp'].values))) - pd.Timedelta('10d')
end_timestamp = datetime.now()
interval = '6h' # "5m","10m","15m","1h","3h","6h","1d","3d","7d"

### Extract data from Signals API

In [9]:
def extract_vessel_positions(list_of_vessels, start_timestamp, end_timestamp, interval):
    URL = "https://api.vortexa.com/v6/signals/vessel-positions"
    start_timestamp_iso = start_timestamp.isoformat()
    end_timestamp_iso = end_timestamp.isoformat()
    ## Make API calls
    PARAMS = {
        'apikey':api_key,
        'time_min':start_timestamp_iso,
        'time_max': end_timestamp_iso,
        'interval': interval,
        'vessel_id': list_of_vessels
    }
    r = requests.get(url = URL, params = PARAMS)
    data = r.json()
    data = pd.DataFrame(data['data'])
    
    # Post-processing
    data['vessel_name'] = data['vessel_id'].map(vessels_dict)
    data.drop_duplicates(subset=['vessel_id','timestamp'],inplace=True)
    data = data.sort_values(by=['timestamp'],ascending=True)
    return data

# Plot animation with gradient color on timestamp
def plot_trail(final_data, save_path=None):
    fig = px.scatter_mapbox(
        final_data, 
        lat="lat", 
        lon="lon", 
        color='vessel_id', 
        size_max=15, 
        zoom=10, 
        animation_frame="timestamp", 
        color_continuous_scale=px.colors.sequential.Viridis,
        text='vessel_name'  # Add vessel names as labels
    )
    
    # Update layout to show the text as labels on the map
    fig.update_traces(marker=dict(size=10), textposition="top right")
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    # Show the plot
    fig.show()
    
    # Save the plot as an HTML file if a save path is provided
    if save_path:
        fig.write_html(save_path)


In [10]:
data = extract_vessel_positions(all_list, start_timestamp, end_timestamp, interval)
data

,lat,lon,vessel_id,timestamp,heading,speed,vessel_name
0,36.508051,-6.617987,3bc8490a0c382976,2024-07-22T00:00:00.000Z,323.0,0.0,NEW ENERGY
400,36.125384,-6.286485,79138d1707099e3e,2024-07-22T00:00:00.000Z,119.0,0.0,ASYA ENERGY
626,24.108015,58.110195,fe16241b4f45168f,2024-07-22T00:00:00.000Z,112.5,16.4,EVEREST ENERGY
170,72.063506,22.505845,72674aadb46c5286,2024-07-22T00:00:00.000Z,72.4,8.2,PIONEER
1,36.510765,-6.614753,3bc8490a0c382976,2024-07-22T06:00:00.000Z,92.0,0.0,NEW ENERGY
...,...,...,...,...,...,...,...
841,56.911525,166.064280,fe16241b4f45168f,2024-09-17T06:00:00.000Z,231.9,11.1,EVEREST ENERGY
843,31.603944,32.302289,72674aadb46c5286,2024-09-17T07:07:59.000Z,41.9,0.0,PIONEER
842,47.111453,-11.049882,3bc8490a0c382976,2024-09-17T07:10:04.000Z,0.0,12.0,NEW ENERGY
845,56.911525,166.064280,fe16241b4f45168f,2024-09-17T07:12:01.000Z,231.9,11.1,EVEREST ENERGY


In [11]:
## Plot vessel movements
plot_trail(data,'lng_plot.html')